In [ ]:
import pandas as pd
import numpy as np

### 프로젝트실습 1
[데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요
이후 QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경하세요

In [ ]:
# selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/kopo_channel_seasonality_new.csv")
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0
...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0
124654,A10,PRODUCT56,201630,275.0
124655,A10,PRODUCT61,201630,0.0
124656,A10,PRODUCT12,201630,15021.0


In [ ]:

selloutData.columns
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(str)
selloutData.QTY = selloutData.QTY.astype(float)
# strColumnList = ["REGIONID","PRODUCT","YEARWEEK"]
# floatColumnList = ["QTY"]

# for i in range(0, len(strColumnList)):
#     eacColumn = strColumnList[i]
#     selloutData[ eacColumn]  = selloutData[ eacColumn ].astype(str)

### 프로젝트 실습2
[불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [ ]:
selloutData['QTY_NEW'] = np.where(selloutData['QTY'] < 0, 0, selloutData['QTY'])

In [ ]:
# selloutData.loc[ selloutData.QTY < 0 ]

### 프로젝트 실습3
[데이터 통합]
selloutData 자료에서
YEAR, WEEK 컬럼을 생성하고 WEEK 컬럼 값이 52 이하인
데이터만 refinedSelloutData 변수에 저장하세요


In [ ]:
# 연도와 주차 컬럼을 생성한다.
selloutData['YEAR'] = selloutData['YEARWEEK'].astype(str).str[:4]
selloutData['WEEK'] = selloutData['YEARWEEK'].astype(str).str[4:]

In [ ]:
refinedSelloutData = selloutData.loc [ selloutData.WEEK <= "52" ]

In [ ]:
selloutData.shape
refinedSelloutData.shape

(123864, 7)

### 프로젝트 실습4
refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요


In [ ]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values(by=sortKey,
                                            ascending=True,
                                            ignore_index=True)
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,01
1,A00,PRODUCT34,201402,679.0,679.0,2014,02
2,A00,PRODUCT34,201403,578.0,578.0,2014,03
3,A00,PRODUCT34,201404,532.0,532.0,2014,04
4,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


### 프로젝트 실습 5
 [지역, 상품, 연도 별 집계]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담으세요

이후 컬럼명을 QTY_MEAN로 변경하세요


In [ ]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
groupData.rename(columns={"mean":"QTY_MEAN"})

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [ ]:
refinedSelloutData.shape

(123864, 7)

### 프로젝트 실습 6 refinedSelloutData와 groupData를
[REGIONID, PRODUCT, YEAR] 키로 조인하여
mergedData 변수에 아래와 같이 담으세요


In [ ]:
joinKey = ["REGIONID", "PRODUCT", "YEAR"]
mergedData = pd.merge(left=refinedSelloutData, right=groupData, on = joinKey, how="inner")
mergedData.shape

(123864, 8)

### 프로젝트 실습 7
mergedData를 활용하여
SEASONALITY = QTY_NEW / QTY_MEAN
SEASONALITY 컬럼을 생성한다

In [ ]:
mergedData = mergedData.rename( columns= {"mean":"QTY_MEAN"})
mergedData["SEASONALITY"] = np.where( mergedData.QTY_MEAN != 0,  mergedData.QTY / mergedData.QTY_MEAN, 0)
mergedData.loc[ (mergedData.REGIONID == "A60") &
                 (mergedData.WEEK == "01") &
                 (mergedData.PRODUCT =="PRODUCT4")]

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
983,A60,PRODUCT4,201401,16.0,16.0,2014,01,29.134615,0.549175
43798,A60,PRODUCT4,201501,23.0,23.0,2015,01,71.711538,0.320729
79174,A60,PRODUCT4,201601,48.0,48.0,2016,01,32.615385,1.471698


In [ ]:
# groupData.loc[ (groupData.REGIONID =="A00") &
#                (groupData.PRODUCT =="PRODUCT34") &
#                (groupData.YEAR == "2014")]

### 프로젝트실습 파이널
mergedData를 활용하여
SEASONALITY = QTY_NEW / QTY_MEAN
SEASONALITY 컬럼을 생성한다.

마지막으로 [지역, 상품, 주차] 별
SEASONALITY 평균 값을
산출하여 finalResult 변수에 담으세요


In [ ]:
groupKey = ["REGIONID","PRODUCT", "WEEK"]
finalResult = mergedData.groupby(by=groupKey)["SEASONALITY"].agg(["mean"]).reset_index()
finalResult

,REGIONID,PRODUCT,WEEK,mean
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192


### 그룹바이 심화

In [ ]:
# 아무생각없이 통계 모델중 이동평균을 데이터에 적용했을때
sortedData["MA5"] = \
    sortedData.QTY_NEW.rolling( window = 5, center=True, min_periods=3).mean()
sortedData["SEASONALITY"] = sortedData.QTY_NEW / sortedData.MA5
groupKey = ["REGIONID","PRODUCT","WEEK"]
myAnswer = sortedData.groupby(by = groupKey)["SEASONALITY"].agg(["mean"]).reset_index()

In [ ]:
# 그룹바이 생각하면 적용했을떄

In [ ]:
# groupData = 데이터프레임.groupby(“그룹키“)

# eachData = groupData.get_group(list(groupData.groups)[0])
# eachData.head()


In [ ]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]
sortedData = refinedSelloutData.sort_values( by = sortKey, ignore_index=True)
groupKey = ["REGIONID","PRODUCT"]

In [ ]:
def rollingMa5(eachGroup):
    # 디버깅 코드
    # groupData = sortedData.groupby( groupKey )
    # eachGroup = groupData.get_group( list (groupData.groups)[0] )
    # 위에 개념이 조금 힘들면 아래로 생각하면 쉽다 왜? 그룹 한개를 가지고 디버깅하는거기 때문에
    # eachGroup = sortedData.loc[ (sqortedData.REGIONID == "A00") &
    #                             (sortedData.PRODUCT == "PRODUCT34")]
    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=1).mean()
    return eachGroup

In [ ]:
groupData = sortedData.groupby( groupKey ).apply ( rollingMa5 )
groupData.reset_index(drop=True)

<ipython-input-21-86862f243f45>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  groupData = sortedData.groupby( groupKey ).apply ( rollingMa5 )


,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MA5
0,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333
1,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000
2,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000
3,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000
4,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000
...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000


In [ ]:
def rollingMa5(eachGroup):

    eachGroup["MA5"] = eachGroup.QTY_NEW.rolling(window=5, center=True, min_periods=3).mean()
    eachGroup["SEASONALITY"] = np.where(eachGroup.MA5 == 0, 0, eachGroup.QTY_NEW / eachGroup.MA5)
    return eachGroup

final = sortedData.groupby( groupKey ).apply(rollingMa5).reset_index(drop=True)

<ipython-input-22-b92c0400b0bf>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final = sortedData.groupby( groupKey ).apply(rollingMa5).reset_index(drop=True)


In [ ]:
groupKey = ["REGIONID","PRODUCT","WEEK"]
seasonFinal = final.groupby(by=groupKey)["SEASONALITY"].agg(["mean"])
seasonFinal.to_csv("season_result.csv")